## SQL Database (SQLite)

In [ ]:
import os, enum
from datetime import datetime
from typing import Optional, Tuple

from sqlalchemy import (
    BigInteger, String, Integer, DateTime, Boolean, Text, JSON, Enum,
    ForeignKey, Index, LargeBinary, select # select for scheduling
)
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy import text as sqltext

from cryptography.fernet import Fernet
import hashlib, hmac, secrets

############################# Database Setup (SQLite + SQLAlchemy Async) ################################

# import models + DB_PATH
from db_models import Base, User, Chat, Message, Config, ChatType, DB_PATH

# Ensure DB directory exists
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

# --- Async engine/session (SQLite over aiosqlite) ---
engine = create_async_engine(f"sqlite+aiosqlite:///{DB_PATH}", echo=False, future=True)
Session = async_sessionmaker(engine, expire_on_commit=False)


################################### App-level encryption key- FERNET ####################################
FERNET_KEY = os.getenv("FERNET_KEY")
if not FERNET_KEY:
    # One-time generate for dev; copy this into an env var before restarting kernels
    FERNET_KEY = Fernet.generate_key().decode()
    print("Generated a dev Fernet key. Save this somewhere safe and set FERNET_KEY env next time:\n", FERNET_KEY)
fernet = Fernet(FERNET_KEY.encode())


# -------------------- DB init --------------------
async def init_db():
    async with engine.begin() as conn:
        # Pragmas: good defaults for SQLite in a concurrent-ish async app
        await conn.execute(sqltext("PRAGMA journal_mode=WAL;"))
        await conn.execute(sqltext("PRAGMA synchronous=NORMAL;"))
        await conn.execute(sqltext("PRAGMA foreign_keys=ON;"))
        await conn.run_sync(Base.metadata.create_all)


######################################################### Some Helpers ############################################################

def _enc(text: Optional[str]) -> Optional[bytes]: # _enc is for encryption; _dec is for decryption as needed
    if not text:
        return None
    if len(text) > 2000:
        text = text[:2000] + " …"
    return fernet.encrypt(text.encode("utf-8"))


def _dec(blob: Optional[bytes]) -> Optional[str]:
    if not blob:
        return None
    return fernet.decrypt(blob).decode("utf-8")


async def upsert_user_and_chat(update) -> Tuple[int, Optional[int], ChatType, Optional[str]]:
    """Ensure a Chat + User row exists; return (chat_id, user_id, chat_type, chat_title)."""
    chat = update.effective_chat
    # v20 returns an enum; guard to string for safety
    t_str = getattr(chat.type, "value", str(chat.type))
    t_map = {
        "private": ChatType.private,
        "group": ChatType.group,
        "supergroup": ChatType.supergroup,
        "channel": ChatType.channel,
    }
    t = t_map.get(t_str, ChatType.private)
    chat_title = getattr(chat, "title", None)

    user = getattr(update, "effective_user", None)
    user_id = getattr(user, "id", None)
    username = getattr(user, "username", None)
    lang = getattr(user, "language_code", None)

    async with Session() as s:
        # chat
        c = await s.get(Chat, chat.id)
        if not c:
            c = Chat(chat_id=chat.id, type=t, title=chat_title)
            s.add(c)
        else:
            c.last_interaction = datetime.utcnow()

        # user
        if user_id:
            now = datetime.utcnow()
            u = await s.get(User, user_id)
            if not u:
                s.add(User(user_id=user_id, username=username, language_code=lang, started_bot=True, last_login_at=now))
            else:
                if u.username != username:
                  u.username = username

                #Every time user interacts, update last_login_at
                u.last_login_at = now  
        await s.commit()
    return chat.id, user_id, t, chat_title

"""Aim of the following part would be to store user messages not directly, but instead 1) their hash/hmac conversions 2) a brief summary of the actual text, after removing PII and then encrypting it using FERNET for privacy concerns."""

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
async def summarize_for_storage(text: str) -> str: #Filhal GPT; later try local model
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    msgs = [
        SystemMessage(content=(
            "You produce a brief, neutral summary for storage.\n"
            "Remove names, phone numbers, emails, addresses, and any identifying info.\n"
            "Max 40 words. No advice, just what the user is talking about."
        )),
        HumanMessage(content=text)
    ]
    out = await model.ainvoke(msgs)
    return out.content.strip()[:300] 


import re
def pii_redact(text: str) -> str: #Kaam Chalau; need to test Presidio PII detector later
    """Redact emails, phone numbers from text."""
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '[email]', text)
    text = re.sub(r'\+?\d[\d\s\-]{7,}\d', '[phone]', text)
    return text

############# Some KEYs to toggle Message features #############

STORE_MESSAGE_BODIES = False   # toggle raw storage
STORE_SUMMARIES      = True  # toggle summary storage

################################################################

# HMAC_KEY = os.getenv("HMAC_KEY") or secrets.token_bytes(32) 
# def hmac_sha256(text: str) -> str:
#     return hmac.new(HMAC_KEY, text.encode("utf-8"), hashlib.sha256).hexdigest()

import hashlib, hmac, secrets
async def log_message(chat_id, user_id, direction, text, rag_used=False, token_usage=None,*, bert_label: str | None=None, bert_conf: float | None= None, gpt_sentiment: str | None=None, final_sentiment: str | None=None, risk_level: str | None=None, suicide_mention: bool | None=None,):
    # compute hash/HMAC for dedup/integrity
    text_hash = hashlib.sha256(text.encode("utf-8")).hexdigest() if text else None #Just SHA256 for now; need to switch to HMAC with a secret key later: "hmac_sha256" function

    # pipeline: (optional) redact -> summarize -> encrypt
    summary_enc = None
    if STORE_SUMMARIES and text:
        redacted = pii_redact(text)
        summary  = await summarize_for_storage(redacted)
        summary_enc = _enc(summary)

    if STORE_MESSAGE_BODIES and text:
        summary_enc = _enc(text)

    async with Session() as s:
        s.add(Message(
            chat_id=chat_id,
            user_id=user_id,
            direction=direction,
            text_hash=text_hash,
            summary_enc=summary_enc,
            rag_used=rag_used,
            token_usage=token_usage,
            bert_label=bert_label,
            bert_conf=bert_conf,
            gpt_sentiment=gpt_sentiment,
            final_sentiment=final_sentiment,
            risk_level=risk_level,
            suicide_mention=suicide_mention,
        ))
        await s.commit()



async def set_config(key: str, value: str):
    async with Session() as s:
        row = await s.get(Config, key)
        if not row:
            from sqlalchemy import insert
            await s.execute(insert(Config).values(key=key, value=value))
        else:
            row.value = value
        await s.commit()

async def get_config(key: str, default: Optional[str] = None) -> Optional[str]:
    async with Session() as s:
        row = await s.get(Config, key)
        return row.value if row else default

# --- Boot the DB once in this kernel ---
import asyncio
await init_db()
print("✅ SQLite ready at:", DB_PATH)

### FastAPI Admin and Ops endpoints

In [ ]:
# FastAPI control plane (admin & ops) – works alongside your polling bot
import os, io, json, logging
from typing import List
from fastapi import FastAPI, Request, UploadFile, File, HTTPException, Depends
from fastapi import Security
from fastapi.security.api_key import APIKeyHeader

from pydantic import BaseModel, Field
from fastapi import Body
from contextlib import asynccontextmanager

from datetime import datetime, timezone, date, time as dt_time
from zoneinfo import ZoneInfo

# Optional API key for safety (send as header: x-api-key)

api_key_header = APIKeyHeader(name="x-api-key", auto_error=False)
now_utc = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
# ADMIN_API_KEY = os.getenv("ADMIN_API_KEY")  # e.g. set in a cell: os.environ["ADMIN_API_KEY"]="dev-secret"

app = FastAPI(title="Fikar Mat - Admin & Ops")

############################### Pydantic Models ##############################

class ThresholdBody(BaseModel):
    value: float = Field(..., ge=0.0, le=1.0, description="Relevance threshold (0..1)")

class FlagsBody(BaseModel):
    ENABLE_RAG: bool | None = Field(None, description="Turn retrieval on/off")
    GENZ_TONE: bool | None = Field(None, description="Use Gen-Z Hinglish tone")

class TestMessageBody(BaseModel):
    chat_id: int | str = Field(..., description="Telegram chat_id to send to")
    text: str = Field("Hello from Admin API 👋", description="Message text")

class ExamDateBody(BaseModel):
    exam_date: date = Field(
        ...,
        description="Exam date (YYYY-MM-DD). Time is assumed to be 09:00 IST (Asia/Kolkata)."
    )

###############################################################################


async def _require_api_key(api_key: str = Security(api_key_header)):
    from fastapi import HTTPException
    if not os.getenv("ADMIN_API_KEY"):  # unlocked in dev
        return
    if api_key != os.getenv("ADMIN_API_KEY"):
        raise HTTPException(status_code=401, detail="invalid api key")
    
def _check_api_key(api_key: str | None):
    cfg = (os.getenv("ADMIN_API_KEY") or "").strip()
    if not cfg:  # dev mode: no key set -> leave open
        return
    if (api_key or "").strip() != cfg:
        raise HTTPException(status_code=401, detail="invalid api key")

@app.get("/")
def root():
    return {"ok": True, "service": "Fikar Mat - Admin API", "try": ["/healthz", "/config", "/docs"]}

@app.get("/healthz")
def healthz():
    
    from datetime import datetime
    return {
        "status": "ok",
        "time": now_utc,
        "kb_loaded": bool(globals().get("Database")),
        "relevance_threshold": globals().get("RELEVANCE_THRESHOLD"),
        "enable_rag": globals().get("ENABLE_RAG"),
        "genz_tone": globals().get("GENZ_TONE"),
    }

@app.get("/config")
def get_config(api_key: str = Security(api_key_header)):
    _check_api_key(api_key)
    return {
        "RELEVANCE_THRESHOLD": globals().get("RELEVANCE_THRESHOLD"),
        "ENABLE_RAG": globals().get("ENABLE_RAG"),
        "GENZ_TONE": globals().get("GENZ_TONE"),
    }

@app.post("/config/rag-threshold")
def set_threshold(body: ThresholdBody = Body(..., examples={"tight": {"summary": "Stricter RAG", "value": {"value": 0.82}}, "loose": {"summary": "Looser RAG", "value": {"value": 0.72}}}), api_key: str = Security(api_key_header)):
    _check_api_key(api_key)
    globals()["RELEVANCE_THRESHOLD"] = float(body.value)
    return {"ok": True, "RELEVANCE_THRESHOLD": body.value}

@app.post("/config/flags")
def set_flags(body: FlagsBody = Body(..., examples={
        "enable_rag": {"summary": "Enable RAG only", "value": {"ENABLE_RAG": True}},
        "toggle_tone": {"summary": "Switch off Gen-Z tone", "value": {"GENZ_TONE": False}},
    }), api_key: str = Security(api_key_header)):
    _check_api_key(api_key)
    if body.ENABLE_RAG is not None:
        globals()["ENABLE_RAG"] = bool(body.ENABLE_RAG)
    if body.GENZ_TONE is not None:
        globals()["GENZ_TONE"] = bool(body.GENZ_TONE)
    return {"ok": True, "ENABLE_RAG": globals()["ENABLE_RAG"], "GENZ_TONE": globals()["GENZ_TONE"]}

@app.post("/kb/reload")
async def kb_reload(file: UploadFile = File(...), api_key: str = Security(api_key_header)):
    _check_api_key(api_key)
    """
    Upload a .txt or .pdf and rebuild FAISS in-memory.
    Replaces the same global `Database` your bot uses.
    """
    raw = await file.read()
    name = (file.filename or "upload").lower()
    ct = (file.content_type or "").lower()

    # Build docs: either from txt or pdf
    from langchain.docstore.document import Document
    docs: List[Document] = []

    if name.endswith(".txt") or ct.startswith("text/"):
        text = raw.decode("utf-8", errors="ignore")
        docs = [Document(page_content=text)]
    elif name.endswith(".pdf"):
        # Try PyPDF loader; fallback to pypdf
        try:
            from langchain_community.document_loaders import PyPDFLoader
            tmp = "/tmp/upload.pdf"
            with open(tmp, "wb") as f:
                f.write(raw)
            loader = PyPDFLoader(tmp)
            docs = loader.load()
        except Exception:
            try:
                import pypdf
                reader = pypdf.PdfReader(io.BytesIO(raw))
                text = "\n".join([(p.extract_text() or "") for p in reader.pages])
                docs = [Document(page_content=text)]
            except Exception:
                raise HTTPException(400, "Install 'langchain-community' or 'pypdf' for PDF support")
    else:
        raise HTTPException(400, "Upload a .txt or .pdf")

    # Split + index (same as your /load)
    from langchain.text_splitter import CharacterTextSplitter
    from langchain.vectorstores import FAISS
    from langchain.embeddings.openai import OpenAIEmbeddings

    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    for i, d in enumerate(chunks):
        d.metadata = {**(d.metadata or {}), "source": name, "chunk": i}

    globals()["Database"] = FAISS.from_documents(chunks, OpenAIEmbeddings())
    logging.info("KB reloaded: %s chunks from %s", len(chunks), name)
    return {"ok": True, "chunks": len(chunks), "source": name}

@app.post("/ops/test-message")
async def send_test_message(body: TestMessageBody = Body(..., examples={
        "example": {"summary": "DM a chat", "value": {"chat_id": 123456789, "text": "Ping from admin 🚀"}}
    }), api_key: str = Security(api_key_header)):
    _check_api_key(api_key)
    """
    Send a test message via the already-built `application` (polling).
    NOTE: Run Cell #2 (start the bot) before calling this endpoint.
    """
    app_bot = getattr(globals().get("application", None), "bot", None)
    if not app_bot:
        raise HTTPException(500, "Telegram application/bot not initialized")
    await app_bot.send_message(chat_id=body.chat_id, text=body.text)
    return {"ok": True}

@app.post("/users/{user_id}/exam-date")
async def set_exam_date(
    user_id: int,
    body: ExamDateBody = Body(..., examples={
        "example": {
            "summary": "Set exam date to 1 March 2026 (09:00 IST)",
            "value": {"exam_date": "2026-03-01"}
        }
    }),
    api_key: str = Security(api_key_header),
):
    """
    Set or update the student's exam date.
    - Input: calendar date (YYYY-MM-DD)
    - Internally: treated as 09:00 Asia/Kolkata, converted to UTC and stored.
    user_id = Telegram user_id / chat_id (for private chats).
    """
    _check_api_key(api_key)

    # 1) Build a local datetime: 09:00 in IST
    local_exam_dt = datetime.combine(body.exam_date, dt_time(hour=9, minute=0))
    ist = ZoneInfo("Asia/Kolkata")

    # 2) Attach IST tz, convert to UTC
    exam_dt_utc = local_exam_dt.replace(tzinfo=ist).astimezone(ZoneInfo("UTC"))

    # 3) Store as naive UTC (matches use of datetime.utcnow())
    exam_dt_utc_naive = exam_dt_utc.replace(tzinfo=None)

    async with Session() as s:
        u = await s.get(User, user_id)
        if not u:
            raise HTTPException(status_code=404, detail=f"user {user_id} not found")

        u.exam_date = exam_dt_utc_naive
        await s.commit()
        await s.refresh(u)

    return {
        "ok": True,
        "user_id": user_id,
        "exam_date": u.exam_date.isoformat(),  # this is UTC
    }




In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
################################# Sentiment Agent (BERT + GPT) #######################################

BERT_MODEL_NAME = "Rahulm94/hing-roberta-mixed-sentiment"

bert_tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
bert_model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME).to(device)
bert_model.eval()

IDX2LABEL = {0: "negative", 1: "neutral", 2: "positive"}

NEG_STRONG = 0.75   # BERT very sure it's negative
CONF_LOW   = 0.55   # BERT unsure
HIGH_RISK_KEYWORDS = ["mar jaa", "marna", "suicide", "khatam", "zindagi khatam"]

## Telegram Bot

In [ ]:
from collections import defaultdict
import os
import logging
from typing import Optional, Tuple
import time, json

from db_models import Base, User, Chat, Message, Config, ChatType, DB_PATH, Score

# Scheduler related imports
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from datetime import timedelta, datetime
import re

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Database = None  # Placeholder for the database object
from dotenv import load_dotenv
load_dotenv()

####################################### ------------------ Globals ------------------ ########################################################################
Database = None  # FAISS vector store
# Simple in-memory conversation history per chat
chat_histories = defaultdict(list)  # chat_id -> [AIMessage/HumanMessage/...]
MAX_TURNS = 8  # keep last 8 messages to stay light
HIGH_RISK_THRESHOLD = 2
high_risk_streaks = defaultdict(int)


# Gen-Z Hinglish style system prompt (tweak as you like)
STYLE_PROMPT = (
    "You are 'Fikar Mat' - a warm, non-judgmental dost. "
    "Talk in light Gen-Z Hinglish (simple Hindi + English), empathetic, concise, "
    "no medical diagnosis. Offer tiny actionable next steps.\n"
    "\n"
    "Most chats are normal ya light stress. For greetings, everyday baatein, "
    "ya halka tension, bas ek normal dost ki tarah casually reply karo.\n"
    "\n"
    "VERY IMPORTANT:\n"
    "- Do NOT randomly mention therapists, doctors, helplines, or 'trusted adults'.\n"
    "- Only talk about professional help or helplines if:\n"
    "    (a) the system message below says the last message is HIGH RISK, OR\n"
    "    (b) the user clearly talks about self-harm, wanting to die, being in danger,\n"
    "        or directly asks for emergency / professional help.\n"
    "Otherwise, keep it light, supportive and friendly."
)

# Safety footer used rarely (optional)
SAFETY_FOOTER = (
    "\n\n(If you ever feel at risk, please reach out to the local helpline Tele-Manas at 14416 ya 1800-891-4416.)"
)

# ---------------- Guardrail config ----------------

DEFAULT_FALLBACK_REPLY = (
    "Yeh topic kaafi sensitive lag raha hai, isliye main thoda extra careful hoon. "
    "Main tere saath hoon baat karne ke liye, lekin saath hi kisi"
    "professional se baat karna bohot zaroori hai."
    + SAFETY_FOOTER
)

# Phrases we explicitly never want to send as-is
BANNED_REPLY_SNIPPETS = [
    "i'll always be here for you no matter what",
    "you don't need anyone else",
    "tumhe kisi aur ki zaroorat nahi",
    "tumhein kisi aur ki zaroorat nahi",
]

# Similarity threshold for when to use RAG context.
# FAISS returns higher score = closer (for cosine). If your score direction is opposite,
# flip the comparator below.
RELEVANCE_THRESHOLD = 0.78  # tune between 0.7–0.85

# --- Admin/ops feature toggles ---
ENABLE_RAG = True
GENZ_TONE  = True

########################################################## -------------- Helpers -------------- ##############################################################################
def _truncate(text: str, max_chars: int = 1500) -> str:
    return text if len(text) <= max_chars else text[:max_chars] + " ..."

async def _fetch_rag_context(query: str, k: int = 2) -> Optional[Tuple[str, float]]:
    """
    Returns (joined_context, best_score) if similar enough, else None.
    Uses similarity_search_with_relevance_scores when available; falls back otherwise.
    """
    global Database

    if not ENABLE_RAG:
        return None
    
    if Database is None:
        return None

    # Prefer API with scores
    if hasattr(Database, "similarity_search_with_relevance_scores"):
        pairs = Database.similarity_search_with_relevance_scores(query, k=k)
        if not pairs:
            logging.info("RAG check | query=%r | no_results", query)
            return None
        # pairs: List[Tuple[Document, score]]

        best_doc, best_score = pairs[0]

        logging.info(
            "RAG check | query=%r | top_score=%.3f | will_use=%s | top_meta=%s",
            query, best_score, best_score >= RELEVANCE_THRESHOLD, getattr(best_doc, "metadata", None)
        )

        if best_score >= RELEVANCE_THRESHOLD:
            ctx = "\n\n".join(_truncate(doc.page_content) for doc, _ in pairs)
            return ctx, best_score
        return None

    # Fallback: no scores – just use top doc as context cautiously
    docs = Database.similarity_search(query, k=k)
    if not docs:
        logging.info("RAG check | query=%r | no_results (fallback)", query)
        return None
    
    logging.info(
        "RAG check | query=%r | score=N/A | top_meta=%s | api=fallback",
        query, getattr(docs[0], "metadata", None)
    )

    ctx = "\n\n".join(_truncate(d.page_content) for d in docs)
    # Without scores, be conservative: only use for *question-like* inputs
    if "?" in query or len(query.split()) >= 5:
        return ctx, 1.0  # pretend strong match
    return None


def _trim_history(chat_id: int):
    msgs = chat_histories[chat_id]
    if len(msgs) > 2 * MAX_TURNS:
        chat_histories[chat_id] = msgs[-2 * MAX_TURNS :]


async def _chat(
    user_text: str,
    chat_id: int,
    rag_context: Optional[str] = None,
    sentiment: str | None = None,
    risk_level: str | None = None,
    suicide_mention: bool | None = None,
) -> str:
    """
    Agent 2: conversational bot.
    Takes into account sentiment/risk flags from Agent 1.
    """
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)  # adjust as needed

    # Build messages
    base_prompt = STYLE_PROMPT if GENZ_TONE else "You are helpful, concise, and neutral."

    # EXTRA safety conditioning for high risk
    extra_safety = ""
    if risk_level == "red" or suicide_mention:
        extra_safety = (
            "\n\nIMPORTANT - CRISIS MODE:\n"
            "- The last message was classified as HIGH RISK for self-harm.\n"
            "- Reply in MAX 80 WORDS.\n"
            "- Main focus: validate feelings + strongly encourage PROFESSIONAL help.\n"
            "- Explicitly mention Tele MANAS as an option in friendly Hinglish, e.g.:\n"
            "  'Main sach mein suggest karunga ki tum kisi professional se baat karo. "
            "Mujhe Tele MANAS ke kaafi ache professionals ka pata hai — agar bolo to "
            "main unka number share kar sakta hoon.'\n"
            "- You may briefly mention talking to a trusted friend/family, but keep it "
            "SECONDARY and low-key (no pressure).\n"
            "- Do NOT give medical or legal advice. Do NOT describe methods of self-harm.\n"
            "- Do NOT include specific phone numbers; the system will send those separately."
        )
    elif risk_level == "amber" or sentiment == "negative":
        extra_safety = (
            "\n\nNote: The user seems low/anxious right now. "
            "Be extra gentle and validating, focus on tiny practical steps."
        )

    system_text = base_prompt + extra_safety
    messages = [SystemMessage(content=system_text)]

    if rag_context:
        messages.append(SystemMessage(content=f"Knowledge base context:\n{rag_context}"))

    # Include short rolling history
    messages.extend(chat_histories[chat_id])

    # Current user message
    messages.append(HumanMessage(content=user_text))

    # Call model
    try:
        start = time.perf_counter()

        ai = await model.ainvoke(messages)
        elapsed_ms = int((time.perf_counter() - start) * 1000)

        reply = ai.content.strip()

        usage = {}
        if hasattr(ai, "response_metadata"):
            usage = ai.response_metadata.get("token_usage", {}) or ai.response_metadata

        logging.info(
            "LLM reply | chat_id=%s | rag_used=%s | latency_ms=%s | usage=%s",
            chat_id, bool(rag_context), elapsed_ms, json.dumps(usage)
        )

        # Update history
        chat_histories[chat_id].append(HumanMessage(content=user_text))
        chat_histories[chat_id].append(AIMessage(content=reply))
        _trim_history(chat_id)

        return reply
    except Exception as e:
        logging.exception("Model call failed: %s", e)
        # Minimal graceful fallback
        return "Arre yaar, thoda glitch ho gaya. Phir se try karoge?"
    
def audit(event: dict, path: str = "events.jsonl"):
    event["ts"] = time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
    try:
        with open(path, "a", encoding="utf-8") as f:
            f.write(json.dumps(event, ensure_ascii=False) + "\n")
    except Exception:
        logging.exception("audit write failed")


def run_bert_sentiment(text: str) -> tuple[str, float, list[float]]:
    """Run BERT once and return (label, confidence, probs)."""
    with torch.no_grad():
        inputs = bert_tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors="pt",
        )
        # move tensors to same device as model
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = bert_model(**inputs)
        logits = outputs.logits[0]  # shape (3,)
        probs = torch.softmax(logits, dim=-1)

        # when converting to Python types, go back to CPU
        probs_list = probs.detach().cpu().tolist()
        idx = int(torch.argmax(probs).item())
        label = IDX2LABEL[idx]
        conf = float(probs_list[idx])

    return label, conf, probs_list

def need_gpt(text: str, bert_label: str, bert_conf: float) -> bool:
    # 1) strong negative -> accept BERT (cheap) unless you *always* want risk
    if bert_label == "negative" and bert_conf >= NEG_STRONG:
        return False

    # 2) low confidence -> ask GPT for help
    if bert_conf < CONF_LOW:
        return True

    # 3) neutral/positive but contains high-risk keyword -> ask GPT
    lower = text.lower()
    if any(kw in lower for kw in HIGH_RISK_KEYWORDS):
        return True

    # 4) else BERT is good enough
    return False

async def call_gpt_classifier(text: str) -> dict:
    """
    GPT-4o-mini classifier:
    returns dict with keys:
    - sentiment: "negative" | "neutral" | "positive"
    - risk_level: "green" | "amber" | "red"
    - suicide_mention: bool
    - emotion: short word like "anxious", "sad"...
    - short_rationale: 1-2 line explanation
    """
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    system_prompt = (
        "You are a safety and sentiment classifier for Hinglish/Hindi/English text.\n"
        "Output STRICT JSON with keys: sentiment, risk_level, suicide_mention, "
        "emotion, short_rationale.\n"
        "- sentiment ∈ {\"negative\", \"neutral\", \"positive\"}\n"
        "- risk_level ∈ {\"green\", \"amber\", \"red\"} where red = high self-harm risk.\n"
        "- suicide_mention is a boolean.\n"
        "No extra text, no markdown, ONLY JSON."
    )

    msgs = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=text),
    ]
    out = await model.ainvoke(msgs)
    raw = out.content.strip()

    # basic robustness
    try:
        data = json.loads(raw)
    except Exception:
        # try to fix common formatting issues
        raw_fixed = raw.strip("`").strip()
        data = json.loads(raw_fixed)

    return data


async def guardrail_filter_reply(user_text: str, raw_reply: str) -> tuple[str, bool, dict]:
    """
    Agent 3: post-hoc safety + PII guardrail on the chatbot reply.

    Returns:
      safe_reply: final text to send to the user
      ok:         whether the original reply was considered safe
      meta:       dict with extra info (reason, etc.)
    """
    reply = (raw_reply or "").strip()

    # --- quick phrase-level sanitisation (string-based) ---
    lower = reply.lower()
    for snippet in BANNED_REPLY_SNIPPETS:
        if snippet in lower:
            # remove the problematic snippet
            reply = re.sub(re.escape(snippet), "", reply, flags=re.IGNORECASE).strip()

    # --- LLM-based safety + PII guardrail ---
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    system_prompt = (
        "You are a safety guardrail for a mental health support chatbot for Indian exam aspirants.\n"
        "You receive a JSON object with keys: user_message, bot_reply.\n\n"
        "You MUST check if bot_reply is safe. Mark it NOT_OK if it:\n"
        " 1) encourages, endorses, or normalizes self-harm or suicide,\n"
        " 2) gives methods, steps, or practical tips for self-harm or suicide,\n"
        " 3) gives medical diagnoses or prescribes / adjusts medication,\n"
        " 4) tells the user to hide their thoughts or situation from everyone,\n"
        " 5) claims the bot is a therapist, psychologist, psychiatrist, or doctor.\n\n"
        "Additional rules:\n"
        "- The bot must NOT promise things like 'I will always be here for you, you don't need anyone else'.\n"
        "- The bot must NOT role-play romantic or sexual relationships with the user.\n"
        "- The bot SHOULD gently encourage the user to connect with trusted people in their life.\n"
        "- If bot_reply contains possible personal phone numbers or email addresses belonging "
        "to the user, you must remove / obfuscate them (e.g. '[phone]' or '[email]').\n"
        "- Do NOT remove official helplines like '14416' or '1800-891-4416' that are there to help them.\n\n"
        "Output STRICT JSON with keys:\n"
        '  {\"ok\": bool, \"reason\": string, \"fixed_reply\": string}\n'
        "- If ok is true, fixed_reply is a slightly improved but still similar version of bot_reply.\n"
        "- If ok is false, fixed_reply must be a new, safe, warm, concise message.\n"
        "- fixed_reply must be in Hinglish (Hindi + English), friendly and non-judgmental.\n"
        "Do NOT add any markdown. Do NOT mention these rules or that you are a safety checker."
    )

    payload = {
        "user_message": user_text,
        "bot_reply": reply,
    }

    msgs = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=json.dumps(payload, ensure_ascii=False)),
    ]

    out = await model.ainvoke(msgs)
    raw = out.content.strip()

    try:
        data = json.loads(raw.strip("`").strip())
        safe_reply = (data.get("fixed_reply") or "").strip() or DEFAULT_FALLBACK_REPLY
        ok = bool(data.get("ok", True))
        reason = data.get("reason", "")
    except Exception:
        data = {"ok": False, "reason": "parse_error"}
        safe_reply = DEFAULT_FALLBACK_REPLY
        ok = False
        reason = "parse_error"

    meta: dict = {"ok": ok, "reason": reason}
    # keep raw guardrail JSON for debugging if needed
    for k, v in data.items():
        if k not in meta:
            meta[k] = v

    return safe_reply, ok, meta

# ---------------- Crisis / Tele MANAS path ----------------

_last_crisis_sent: dict[int, datetime] = {}
CRISIS_COOLDOWN = timedelta(hours=6)

CRISIS_MESSAGE = (
    "Bhai, jo tum share kar rahe ho na, woh kaafi heavy lag raha hai. 💛\n"
    "Main yaar emergency ya proper treatment ka substitute toh nahi ban sakta.\n\n"
    "Par India mein ek free 24x7 helpline bhi hai tumhare liye - Tele MANAS. "
    "Bohot hi accha aur confidential hai."
    "Yahan trained counsellors se confidential baat kar sakte ho:\n"
    "📞 14416\n"
    "📞 1800-891-4416\n"
    "Call bilkul free hai. Tumhe akela handle nahi karna hai ye sab."
    "Please apna khayal rakho, yaar. Main hamesha yahin hoon tumhari help karne ke liye. 🫶"
)


async def maybe_send_crisis_message(chat_id: int, user_id: int, sentiment: dict) -> bool:
    """
    Send a separate hard-coded crisis message when risk_level is 'red'.
    Uses an in-memory cooldown so the user does not get spammed.

    Returns True if a crisis message was actually sent.
    """
    if sentiment.get("risk_level") != "red":
        return False

    now = datetime.utcnow()
    last = _last_crisis_sent.get(user_id)
    if last and (now - last) < CRISIS_COOLDOWN:
        return False  # still within cooldown

    bot = getattr(globals().get("application", None), "bot", None)
    if not bot:
        logging.warning("Crisis: application.bot not ready, skipping crisis message")
        return False

    try:
        await bot.send_message(chat_id=chat_id, text=CRISIS_MESSAGE)
        await log_message(
            chat_id=chat_id,
            user_id=user_id,
            direction="out",
            text="crisis-tele-manas-message",
            rag_used=False,
            token_usage=None,
            final_sentiment=sentiment.get("final_sentiment"),
            risk_level=sentiment.get("risk_level"),
            suicide_mention=sentiment.get("suicide_mention"),
        )
        _last_crisis_sent[user_id] = now
        return True
    except Exception:
        logging.exception("Failed to send crisis message to %s", user_id)
        return False


######################################### Nudges via APScheduler ##############################################################

async def send_last_login_nudges():
    """
    If a student hasn't logged in for ~6 days, send a gentle 'missing you' ping.
    Runs daily; only fires once per ~week gap per user.
    """
    from datetime import datetime, timedelta

    now = datetime.utcnow()
    six_days_ago = now - timedelta(days=6)
    seven_days_ago = now - timedelta(days=7)

    async with Session() as s:
        stmt = select(User).where(
            User.last_login_at.is_not(None),
            User.last_login_at <= six_days_ago,
            User.last_login_at > seven_days_ago,
        )
        res = await s.execute(stmt)
        users = res.scalars().all()

    if not users:
        return

    bot = getattr(globals().get("application", None), "bot", None)
    if not bot:
        logging.warning("Scheduler: application.bot not ready, skipping last-login nudges")
        return

    text = (
        "Sab thik toh hai na? 😊\n"
        "Is week humari baat nahi hui ab tak. "
        "Jab mann kare, aa jaa baat karne. Main yahin hoon."
    )

    for u in users:
        try:
            await bot.send_message(chat_id=u.user_id, text=text)
            await log_message(chat_id=u.user_id, user_id=u.user_id,
                              direction="out", text="last-login-nudge")
        except Exception:
            logging.exception("Failed to send last-login nudge to %s", u.user_id)


async def send_exam_nudges():
    """
    Countdown nudges relative to User.exam_date (3m, 2m, 1m, 1w, 1d).
    """
    from datetime import datetime

    now = datetime.utcnow()

    async with Session() as s:
        stmt = select(User).where(User.exam_date.is_not(None))
        res = await s.execute(stmt)
        users = res.scalars().all()

    if not users:
        return

    bot = getattr(globals().get("application", None), "bot", None)
    if not bot:
        logging.warning("Scheduler: application.bot not ready, skipping exam nudges")
        return

    for u in users:
        days_left = (u.exam_date - now).days
        if days_left not in (90, 60, 30, 7, 1):
            continue

        if days_left == 90:
            line = (
                "3 months left to prepare for your exam! 📚\n"
                "Kuch planning-shanning start karni ho toh batao — "
                "relaxation exercises, breathing exercises, kuch bhi!"
            )
        elif days_left == 60:
            line = (
                "2 months left for your exam! 💡\n"
                "Ab thoda structured plan bana sakte hain. "
                "Main saath hoon agar tension ho."
            )
        elif days_left == 30:
            line = (
                "1 month baaki hai tere exam ke! 💪\n"
                "Jo bhi stress ya confusion ho, mujhe bol — "
                "saath mein strategy bana lenge."
            )
        elif days_left == 7:
            line = (
                "1 week left! ✨\n"
                "Ab sirf smart revision, achchi neend aur thoda self-care. "
                "Tu handle kar lega."
            )
        else:  # days_left == 1
            line = (
                "Kal tera exam hai! 🫶\n"
                "Deep breath, tu ready hai. Agar anxiety ho, idhar rant kar de."
            )

        try:
            await bot.send_message(chat_id=u.user_id, text=line)
            await log_message(chat_id=u.user_id, user_id=u.user_id,
                              direction="out", text="exam-nudge")
        except Exception:
            logging.exception("Failed to send exam nudge to %s", u.user_id)


async def get_score_trend(user_id: int) -> str:
    """
    Look at the last up-to-6 months of scores for this user and return:
      - "High" = trend going up
      - "Low"  = trend going down
      - "Same" = roughly stable

    Simple heuristic:
    - Take up to 6 latest months
    - Compare first vs last score
    - If diff >= +5 points → "High"
      diff <= -5 points → "Low"
      otherwise         → "Same"
    """
    from datetime import date
    async with Session() as s:
        stmt = (
            select(Score)
            .where(Score.user_id == user_id)
            .order_by(Score.month.desc())
            .limit(6)
        )
        res = await s.execute(stmt)
        rows = list(res.scalars().all())

    # Not enough data → treat as neutral
    if len(rows) < 3:
        return "Same"

    # We got latest-first; reverse to oldest-first
    rows = list(reversed(rows))
    scores = [r.avg_score for r in rows]

    first = scores[0]
    last = scores[-1]
    diff = last - first

    THRESHOLD = 5.0  # you can tweak this

    if diff >= THRESHOLD:
        return "High"
    elif diff <= -THRESHOLD:
        return "Low"
    else:
        return "Same"


async def send_score_nudges():
    """
    Monthly score check-in:
    - Prefer sending in the last week of the month (day >= 25)
    - Only if at least ~30 days since last_score_nudge_at
    - Message depends on score trend: High / Same / Low
    """
    from datetime import datetime, timedelta

    now = datetime.utcnow()

    # Prefer nudges near the end of the month
    # if now.day < 25:
    #     return

    async with Session() as s:
        stmt = select(User)
        res = await s.execute(stmt)
        users = res.scalars().all()

        bot = getattr(globals().get("application", None), "bot", None)
        if not bot:
            logging.warning("Scheduler: application.bot not ready, skipping score nudges")
            return

        for u in users:
            # Only once per ~month
            if u.last_score_nudge_at and (now - u.last_score_nudge_at).days < 30:
                continue

            trend = (await get_score_trend(u.user_id)) or "Same"
            trend_norm = trend.strip().lower()

            if trend_norm == "low":
                msg = (
                    "Bhai, tera score thoda low ho raha hai last months se. 💭\n"
                    "Sab theek hai? Agar kuch hua hai ya feel ho raha ho, "
                    "mujhe bata sakta hai — main sunne ke liye yahin hoon."
                )
            elif trend_norm == "high":
                msg = (
                    "Sabaash bhai! Scores upar ki taraf jaa rahe hain. 🔥\n"
                    "Tu hai aag, Milkha! Aise hi consistency bana ke rakh."
                )
            else:  # 'same' or unknown
                msg = (
                    "Sahi jaa raha hai yaar — score trend kaafi stable lag raha hai. 😊\n"
                    "Bas thoda aur push karna hai, tu kar sakta hai."
                )

            try:
                await bot.send_message(chat_id=u.user_id, text=msg)
                await log_message(chat_id=u.user_id, user_id=u.user_id,
                                  direction="out", text=f"score-nudge-{trend_norm}")
                u.last_score_nudge_at = now
            except Exception:
                logging.exception("Failed to send score nudge to %s", u.user_id)

        await s.commit()


####################################################### Sentiment Agent #########################################################################################
async def run_sentiment_agent(chat_id: int, user_id: int, text: str) -> dict:
    """
    Agent 1: hybrid sentiment + risk classifier.
    - Always runs BERT.
    - Optionally calls GPT if gating says so.
    - Returns a dict and also logs the *incoming* message with sentiment metadata.
    """
    bert_label, bert_conf, bert_probs = run_bert_sentiment(text)

    use_gpt = need_gpt(text, bert_label, bert_conf)

    gpt_data = None
    final_sentiment = bert_label
    risk_level = "green"
    suicide_mention = False
    gpt_sentiment = None

    if use_gpt:
        try:
            gpt_data = await call_gpt_classifier(text)
            gpt_sentiment = gpt_data.get("sentiment")
            final_sentiment = gpt_sentiment or bert_label
            risk_level = gpt_data.get("risk_level", "green")
            suicide_mention = bool(gpt_data.get("suicide_mention", False))
        except Exception:
            logging.exception("GPT classifier failed, falling back to BERT-only")
    else:
        # BERT-only path, you can still add simple rules for risk here if you want
        risk_level = "green"
        suicide_mention = False

    # Log this incoming message with full sentiment metadata
    await log_message(
        chat_id=chat_id,
        user_id=user_id,
        direction="in",
        text=text,
        rag_used=False,
        token_usage=None,
        bert_label=bert_label,
        bert_conf=bert_conf,
        gpt_sentiment=gpt_sentiment,
        final_sentiment=final_sentiment,
        risk_level=risk_level,
        suicide_mention=suicide_mention,
    )

    return {
        "bert_label": bert_label,
        "bert_conf": bert_conf,
        "gpt_sentiment": gpt_sentiment,
        "final_sentiment": final_sentiment,
        "risk_level": risk_level,
        "suicide_mention": suicide_mention,
        "gpt_raw": gpt_data,
    }



######################################################## -------------- Handlers --------------##############################################################################

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Start: greet + upsert + minimal audit."""
    chat_id, user_id, *_ = await upsert_user_and_chat(update)
    msg = (
        "Namaste dost! Main Fikar Mat 🤝\n"
        "Seedha bol  — dimaag mein jo bhi chal raha hai. "
        "Bas bolo judgement zero."
    )
    await log_message(chat_id, user_id, "out", "welcome-start")
    await context.bot.send_message(chat_id=chat_id, text=msg)

async def load(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Load and index your local txt as a small knowledge base for RAG.
    """
    global Database
    try:
        loader = TextLoader("fikar_mat_anxiety_toolkit.txt", encoding="utf-8")
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_documents(documents)

        for i, d in enumerate(docs):
            d.metadata = {**(d.metadata or {}), "source": "fikar_mat_anxiety_toolkit.txt", "chunk": i}
        
        Database = FAISS.from_documents(docs, OpenAIEmbeddings())

        logging.info("KB loaded: %s chunks from %s", len(docs), "fikar_mat_anxiety_toolkit.txt")

        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="Knowledge base load ho gaya ✅  (RAG ready!)",
        )
    except Exception as e:
        logging.exception("Load failed: %s", e)
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="Arre, file load nahi ho paayi. Filename/encoding check karoge?",
        )

async def reset(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Reset short-term in-memory history (your existing behavior) + log."""
    chat_id, user_id, *_ = await upsert_user_and_chat(update)
    chat_histories.pop(chat_id, None)
    await log_message(chat_id, user_id, "out", "memory-reset")
    await context.bot.send_message(chat_id=chat_id, text="Memory reset. Fresh start ✨")

async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Default message handler:
      1) upsert user/chat
      2) run sentiment agent (Agent 1)
      3) optional RAG lookup
      4) call chat agent (Agent 2) with sentiment/risk info
      5) log + reply
    """
    chat_id, user_id, *_ = await upsert_user_and_chat(update)
    user_text = (update.message.text or "").strip()

    # 1. Run sentiment agent
    sentiment = await run_sentiment_agent(chat_id=chat_id, user_id=user_id, text=user_text)

    # Track consecutive high-risk (red / suicidal) messages
    global high_risk_streaks

    is_high_risk = (sentiment["risk_level"] == "red") or bool(sentiment["suicide_mention"])
    if is_high_risk:
        high_risk_streaks[chat_id] += 1
    else:
        high_risk_streaks[chat_id] = 0

    crisis_count = high_risk_streaks[chat_id]

    # await log_message(chat_id, user_id, "in", user_text)

    rag_ctx = None
    rag_used = False
    try:
        rag_hit = await _fetch_rag_context(user_text, k=2)
        if rag_hit:
            rag_ctx, score = rag_hit
            rag_used = True
            logging.info("RAG hit with score=%.3f", score)
    except Exception as e:
        logging.exception("RAG lookup failed: %s", e)
    
    audit({
    "type": "user_msg",
    "chat_id": chat_id,
    "text": user_text[:200],  # keep short in logs
    "rag_used": rag_used,
    "final_sentiment": sentiment["final_sentiment"],
    "risk_level": sentiment["risk_level"],
    })

    # 2. Call chat agent -> raw reply
    raw_reply = await _chat(
        user_text=user_text,
        chat_id=chat_id,
        rag_context=rag_ctx,
        sentiment=sentiment["final_sentiment"],
        risk_level=sentiment["risk_level"],
        suicide_mention=sentiment["suicide_mention"],
    )

    # 3. Guardrail agent on the reply (safety + PII)
    safe_reply, ok, gr_meta = await guardrail_filter_reply(
        user_text=user_text,
        raw_reply=raw_reply,
    )

    audit({
        "type": "bot_reply",
        "chat_id": chat_id,
        "ok": ok,
        "guardrail_reason": gr_meta.get("reason"),
        "rag_used": rag_used,
        "final_sentiment": sentiment["final_sentiment"],
        "risk_level": sentiment["risk_level"],
    })

    # 4. Log + send final safe reply (not the raw one)
    await log_message(
        chat_id,
        user_id,
        "out",
        safe_reply,
        rag_used=rag_used,
        token_usage=None,
        final_sentiment=sentiment["final_sentiment"],
        risk_level=sentiment["risk_level"],
        suicide_mention=sentiment["suicide_mention"],
    )

    prefix = "📚 " if rag_used else ""
    await context.bot.send_message(chat_id=chat_id, text=prefix + safe_reply)

    # 5. Separate crisis path for repeated / strong 'red' risk
    if is_high_risk and crisis_count >= HIGH_RISK_THRESHOLD:
        await maybe_send_crisis_message(
            chat_id=chat_id,
            user_id=user_id,
            sentiment=sentiment,
        )



#################################-----------------App Wiring------------------###############################################

application = ApplicationBuilder().token(os.getenv("TELEGRAM_BOT_TOKEN")).build()
application.add_handler(CommandHandler("start", start))
application.add_handler(CommandHandler("load", load))
application.add_handler(CommandHandler("reset", reset))
application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))

# ---------------- Scheduler setup (jobs are started in Cell 6) ----------------

scheduler = AsyncIOScheduler(timezone="UTC")

# Run once per day; internal logic decides whether to actually send something
scheduler.add_job(send_last_login_nudges, "cron", hour=10, minute=0) # means 10 am UTC -> 15:30 IST
scheduler.add_job(send_exam_nudges,       "cron", hour=11, minute=0)
scheduler.add_job(send_score_nudges,      "cron", hour=18, minute=0)

logging.info("APScheduler configured with nudge jobs (not started yet).")

### FastAPI - Run server/ STOP

In [ ]:
# Managed Uvicorn server you can start/stop from the notebook
import os, threading, nest_asyncio, uvicorn
from uvicorn import Config, Server
nest_asyncio.apply()

PORT = int(os.getenv("PORT", 8000))

# Build a server object we can signal to stop later
_uvicorn_config = Config(app, host="0.0.0.0", port=PORT, log_level="info")
UVICORN_SERVER = Server(_uvicorn_config)

def _run_server():
    UVICORN_SERVER.run()

API_THREAD = threading.Thread(target=_run_server, daemon=True)
API_THREAD.start()
print(f"Admin API running at http://localhost:{PORT}  (use header x-api-key if set)")


In [9]:
# Gracefully stop the managed Uvicorn server
try:
    UVICORN_SERVER.should_exit = True
    API_THREAD.join(timeout=5)
    print("Admin API stopped.")
except NameError:
    print("No running server/thread found. (If you used uvicorn.run(...), restart the kernel.)")


Admin API stopped.


### BOT START

In [ ]:
await application.initialize()
await application.start()
await application.updater.start_polling()
print("✅ Bot started. Send /start to your bot in Telegram.")

# Start the scheduler once the bot is ready
try:
    scheduler.start()
    print("✅ APScheduler started with nudge jobs.")
except Exception as e:
    print("Scheduler already running or failed:", e)

print("✅ Bot started. Send /start to your bot in Telegram.")

### Execute the Cell Below for Manual testing of the Triggers only

In [ ]:
# Manual test: trigger the nudge jobs once
await send_last_login_nudges()
await send_exam_nudges()
await send_score_nudges()

### Stopping the Bot

In [ ]:
await application.updater.stop()
await application.stop()
await application.shutdown()
print("Bot stopped.")

# Try to stop scheduler too
try:
    scheduler.shutdown(wait=False)
    print("Scheduler stopped.")
except NameError:
    print("Scheduler not defined in this session.")
except Exception as e:
    print("Error stopping scheduler:", e)